In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")
import os
import pathlib
import random 
import argparse
from scipy import optimize

inputdir = "/Volumes/HNWD02/outdir/output_methyl_ReadBased_Models/01_output/TSMA_panel"
path_to_all_fa = "/Volumes/HNSD01/storage/ref/hg19"
outdir = "/Volumes/HNSD01/outdir"
outputdir = os.path.join(outdir, "output_methyl_ReadBased_Models")
path_to_01_output = os.path.join(outputdir, "01_output")
path_to_02_output = os.path.join(outputdir, "02_output")

metadata = pd.read_excel("/Users/hieunguyen/src/tsma_micro_atlas/metadata/metadata_TSMA_R8281_R8282.xlsx")
paneldf = pd.read_excel("./assets/12967_2024_5416_MOESM1_ESM.xlsx")
all_regions = paneldf.Region_name.unique()

path_to_03_output = os.path.join(outputdir, "03_output")
path_to_04_output = os.path.join(outputdir, "04_output")
os.system(f"mkdir -p {path_to_04_output}")

sum_preddf = pd.DataFrame(
    data = ['Breast', 'CRC', 'Gastric', 'Liver', 'Lung', 'WBC', 'no data'],
    columns = ['prediction']
)
for sampleid in metadata.LABCODE.unique():
    all_preds = [
        item for item in pathlib.Path(path_to_03_output).glob(f"*{sampleid}/microAtlas_data/region_*/*_prediction_on_*.xlsx")
    ]

    predictiondf = pd.DataFrame()
    for file in tqdm(all_preds):
        tmpdf = pd.read_excel(file, index_col = [0])
        predictiondf = pd.concat([predictiondf, tmpdf], axis = 0)
    predictiondf = predictiondf.reset_index(drop = True)
    predictiondf.to_excel(os.path.join(path_to_04_output, f"prediction_{sampleid}.xlsx"))
    tmp_sum_preddf = predictiondf.groupby("prediction")["loocv_val_sample"].count().reset_index()
    tmp_sum_preddf.columns = ["prediction", sampleid]   
    sum_preddf = sum_preddf.merge(tmp_sum_preddf, right_on = "prediction", left_on = "prediction", how = "left")

 94%|█████████▍| 2716/2892 [00:07<00:00, 365.39it/s]